***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-febe  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install nltk==3.7

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 13 11:21 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'razm_bucket2' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)

In [ ]:
doc_text_pairs = parquetFile.select("id","text").rdd


In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)

def tokenize(text):
    """
    this method creates a list of tokens from a given text + removes stop words
    :param text: text
    :return: list of tokens
    """

    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    new_tokens = []
    for token in tokens:
        if token in all_stopwords:
            continue
        else:
            new_tokens.append(token)
    return new_tokens

In [ ]:
def word_count(id,text):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

  Dict={}
  lst=[]
  to_return={}
  
  # YOUR CODE HERE
  Dict=Counter(tokens)
  for item in Dict:
    if item in all_stopwords:
      continue
    else:
      x=(id,Dict[item])
      to_return[item]=x

  lst=list(to_return.items())
  return lst;


In [ ]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  sorted_list = sorted(unsorted_pl, key=lambda tup: tup[0])
  return sorted_list

In [ ]:
import math
def build_norm_doc(tok_text):
    temp_dict = {}
    for term in list(tok_text):
        if term not in temp_dict:
            temp_dict[term] = 0
        temp_dict[term] +=1
    sum = 0
    for term in temp_dict:
        sum += temp_dict[term]**2
    if sum == 0:
        return sum
    return 1/math.sqrt(sum)

In [ ]:
client = storage.Client()
bucket = client.bucket(bucket_name)

In [ ]:
#write norm dict to bucket
doc_text_pairs_new = doc_text_pairs.mapValues(tokenize).mapValues(build_norm_doc)
norm_dict= doc_text_pairs_new.collectAsMap()
x="norm_dict"
with open(f"{x}.pickle", "wb") as f:
    pickle.dump(norm_dict, f)
blob_title_dict = bucket.blob(f"postings_gcp/{x}.pickle")
blob_title_dict.upload_from_filename(f"{x}.pickle")

In [ ]:
# write title dict to bucket: 
title_dict=dict(doc_title_pairs.collectAsMap())
x="title_dict"
with open(f"{x}.pickle", "wb") as f:
    pickle.dump(title_dict, f)
blob_title_dict = bucket.blob(f"postings_gcp/{x}.pickle")
blob_title_dict.upload_from_filename(f"{x}.pickle")


In [ ]:
# write DL dict to bucket:
DL= doc_text_pairs.map(lambda x: (x[0], len(tokenize(x[1])))).collectAsMap()
x="DL"
with open(f"{x}.pickle", "wb") as f:
    pickle.dump(DL, f)
blob_DL_dict = bucket.blob(f"postings_gcp/{x}.pickle")
blob_DL_dict.upload_from_filename(f"{x}.pickle")

In [ ]:
def f_sum(x):
  c=0
  for i in x:
    c += i
  return c

def calculate_term_total(postings):
  return postings.mapValues(lambda x: [y[1] for y in x]).mapValues(f_sum)

In [ ]:
# write term total dict to bucket:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
term_total = calculate_term_total(postings).groupByKey().mapValues(f_sum).collectAsMap()
x="term_total"
with open(f"{x}.pickle", "wb") as f:
    pickle.dump(term_total, f)
blob_term_total_dict = bucket.blob(f"postings_gcp/{x}.pickle")
blob_term_total_dict.upload_from_filename(f"{x}.pickle")